<a href="https://colab.research.google.com/github/yakovsushenok/Thesis/blob/main/BirdSongIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os
import pandas as pd
import math, random
import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
from google.colab import drive
from sklearn.model_selection import StratifiedShuffleSplit
drive.mount('/content/gdrive')
import time
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
try:
    from scipy.fftpack import fft, ifft
except ImportError:
    from numpy.fft import fft, ifft
from scipy.signal import lfilter
import scipy.io as sio
from scipy import signal

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Data

In [2]:
# extracting the training data (audio files) from the zip file (12 minutes)
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/train_short_audio.zip', 'r')
zip_ref.extractall('/content/tmp') 
zip_ref.close()

In [2]:
# extracting the data which has audio files that are of similar length to the testing data
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/train_soundscapes.zip', 'r')
zip_ref.extractall('/content/trainSoundscapes') 
zip_ref.close()

df = pd.read_csv("/content/gdrive/MyDrive/train_metadata.csv") # the metadata
df['relative_path'] = '/content/tmp/' + df['primary_label'] + '/' + df['filename'] 

### Subsetting the data so that we're left with subspecies which have 300+ samples and with audio rated 4.0+

In [3]:
df = df[(df['primary_label'].value_counts().reindex(df['primary_label'])>299).values & (df['rating'] > 3.5)]
df = df[['relative_path', 'primary_label']]
unique_labels = df.primary_label.unique()
mapping = dict(zip(unique_labels, range(len(unique_labels))))
df.primary_label = df.primary_label.map(mapping)

### We're going to partition our data into a training/validation/testing test with 80% being the training, 10% for validating and 10% for testing with each species category having the same distribution.

In [4]:
# splitting into train, val+test
X, y = df['relative_path'], df['primary_label']
split1 = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=0)
train_index, val_test = next(split1.split(X, y))
# splitting into val, test
X1, y1 = df.iloc[val_test, 0], df.iloc[val_test, 1]
split2 = StratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=0)
val_index, test_index = next(split2.split(X1, y1))
# subsetting the datasets
df_train = df.iloc[train_index,:]
df_val = df.iloc[val_index, :]
df_test = df.iloc[test_index, :]

# Utility classes

In [5]:
### MRCG CODE
epsc = 0.000001

def mrcg_extract(sig, sampFreq = 32000): # Sample frequency is always 32,000 in our case
    # Code From: https://github.com/MoongMoong/MRCG_python/blob/master/MRCG_python_master/mrcg/MRCG.py
    
    beta = 1000 / np.sqrt(sum(map(lambda x:x*x,sig)) / len(sig))
    sig = sig*beta
    sig = sig.reshape(len(sig), 1)
    g = gammatone(sig, 64, sampFreq)
    cochlea1 = np.log10(cochleagram(g, int(sampFreq * 0.025), int(sampFreq * 0.010)))
    cochlea2 = np.log10(cochleagram(g, int(sampFreq * 0.200), int(sampFreq * 0.010)))
    cochlea1 = cochlea1[:,:]
    cochlea2 = cochlea2[:,:]
    cochlea3 = get_avg(cochlea1, 5, 5)
    cochlea4 = get_avg(cochlea1, 11, 11)
    
    all_cochleas = np.concatenate([cochlea1,cochlea2,cochlea3,cochlea4],0)
    del0 = deltas(all_cochleas)
    ddel = deltas(deltas(all_cochleas, 5), 5)

    ouotput = np.concatenate((all_cochleas, del0, ddel), 0)

    return ouotput

def gammatone(insig, numChan=128, fs = 16000): 
    fRange = [50, 8000]
    filterOrder = 4
    gL = 2048
    sigLength = len(insig)
    phase = np.zeros([numChan, 1])
    erb_b = hz2erb(fRange)

    
    erb_b_diff = (erb_b[1]-erb_b[0])/(numChan-1)
    erb = np.arange(erb_b[0], erb_b[1]+epsc, erb_b_diff)
    cf = erb2hz(erb)
    b = [1.019 * 24.7 * (4.37 * x / 1000 + 1) for x in cf]
    gt = np.zeros([numChan, gL])
    tmp_t = np.arange(1,gL+1)/fs
    for i in range(numChan):
        gain = 10**((loudness(cf[i])-60)/20)/3*(2 * np.pi * b[i] / fs)**4
        tmp_temp = [gain*(fs**3)*x**(filterOrder - 1)*np.exp(-2 * np.pi * b[i] * x)*np.cos(2 * np.pi * cf[i] * x + phase[i]) for x in tmp_t]
        tmp_temp2 = np.reshape(tmp_temp, [1, gL])

        gt[i, :] = tmp_temp2

    sig = np.reshape(insig,[sigLength,1])
    gt2 = np.transpose(gt)
    resig = np.matlib.repmat(sig,1,numChan)
    r = np.transpose(fftfilt(gt2,resig,numChan))
    return r

def hz2erb(hz):  
    erb1 = 0.00437
    erb2 = np.multiply(erb1,hz)
    erb3 = np.subtract(erb2,-1)
    erb4 = np.log10(erb3)
    erb = 21.4 *erb4
    return erb

def erb2hz(erb): 
    hz = [(10**(x/21.4)-1)/(0.00437) for x in erb]
    return hz

def loudness(freq): 
    dB=60
    fmat = sio.loadmat('/content/gdrive/MyDrive/f_af_bf_cf.mat')
    af = fmat['af'][0]
    bf = fmat['bf'][0]
    cf = fmat['cf'][0]
    ff = fmat['ff'][0]
    i = 0
    while ff[i] < freq:
        i = i + 1

    afy = af[i - 1] + (freq - ff[i - 1]) * (af[i] - af[i - 1]) / (ff[i] - ff[i - 1])
    bfy = bf[i - 1] + (freq - ff[i - 1]) * (bf[i] - bf[i - 1]) / (ff[i] - ff[i - 1])
    cfy = cf[i - 1] + (freq - ff[i - 1]) * (cf[i] - cf[i - 1]) / (ff[i] - ff[i - 1])
    loud = 4.2 + afy * (dB - cfy) / (1 + bfy * (dB - cfy))
    return loud

def fftfilt(b,x,nfft): 
    fftflops = [18, 59, 138, 303, 660, 1441, 3150, 6875, 14952, 32373, 69762,
                149647, 319644, 680105, 1441974, 3047619, 6422736, 13500637, 28311786,
                59244791, 59244791*2.09]
    nb, _ = np.shape(b)
    nx, mx = np.shape(x)
    n_min = 0
    while 2**n_min < nb-1:
        n_min = n_min+1
    n_temp = np.arange(n_min, 21 + epsc, 1)
    n = np.power(2,n_temp)
    fftflops = fftflops[n_min-1:21]
    L = np.subtract(n,nb-1)
    lenL= np.size(L)
    temp_ind0 = np.ceil(np.divide(nx,L))
    temp_ind = np.multiply(temp_ind0,fftflops)
    temp_ind = np.array(temp_ind)
    ind = np.argmin(temp_ind)
    nfft=int(n[ind])
    L=int(L[ind])
    b_tr = np.transpose(b)
    B_tr = fft(b_tr,nfft)
    B = np.transpose(B_tr)
    y = np.zeros([nx, mx])
    istart = 0
    while istart < nx :
        iend = min(istart+L,nx)
        if (iend - istart) == 1 :
            X = x[0][0]*np.ones([nx,mx])
        else :
            xtr = np.transpose(x[istart:iend][:])
            Xtr = fft(xtr,nfft)
            X = np.transpose(Xtr)
        temp_Y = np.transpose(np.multiply(B,X))
        Ytr = ifft(temp_Y,nfft)
        Y = np.transpose(Ytr)
        yend = np.min([nx, istart + nfft])
        y[istart:yend][:] = y[istart:yend][:] + np.real(Y[0:yend-istart][:])

        istart = istart + L
    
    return y

def cochleagram(r, winLength = 320, winShift=160): 
    numChan, sigLength = np.shape(r)
    increment = winLength / winShift
    M = np.floor(sigLength / winShift)
    a = np.zeros([numChan, int(M)])
    rs = np.square(r)
    rsl = np.concatenate((np.zeros([numChan,winLength-winShift]),rs),1)
    for m in range(int(M)):
        temp = rsl[:,m*winShift : m*winShift+winLength]
        a[:, m] = np.sum(temp,1)

    return a

def get_avg( m , v_span, h_span): 
    nr,nc = np.shape(m)

    fil_size = (2 * v_span + 1) * (2 * h_span + 1)
    meanfil = np.ones([1+2*h_span,1+2*h_span])
    meanfil = np.divide(meanfil,fil_size)

    out = signal.convolve2d(m, meanfil, boundary='fill', fillvalue=0, mode='same')
    return out

def deltas(x, w=9) : 
    nr,nc = np.shape(x)
    if nc ==0 :
        d= x
    else :
        hlen = int(np.floor(w / 2))
        w = 2 * hlen + 1
        win=np.arange(hlen, int(-(hlen+1)), -1)
        temp = x[:, 0]
        fx = np.matlib.repmat(temp.reshape([-1,1]), 1, int(hlen))
        temp = x[:, nc-1]
        ex = np.matlib.repmat(temp.reshape([-1,1]), 1, int(hlen))
        xx = np.concatenate((fx, x, ex),1)
        d = lfilter(win, 1, xx, 1)
        d = d[:,2*hlen:nc+2*hlen]

    return d

Now that I have the MRCG code, I want to create a dataset which will hold these MRCG values. This will mean I'll not have to pre-process them every run of the model.

In [6]:
def get_mrcg_from_file(file):
  
  MAX = 75348463 # zero padding so that the length is equal to the length of the longest waveform 
  print("iii")
  waveform, sr = torchaudio.load(file)
  target = torch.zeros(MAX)
  source = waveform[0]
  target[:len(source)] = source
  return mrcg_extract(target)



In [ ]:
df_toy = df[int(0.25*len(df)):int(0.5*len(df))]
df_toy['mrcg'] = df_toy['relative_path'].apply(get_mrcg_from_file)
max = 0
for i in range(len(df_toy)):
  if max < df_toy.iloc[i,-1].shape[0]:
    max = df_toy.iloc[i,-1].shape[0]
print(max) # 73084832 (0:0.25, ), 75348463 (0.25:0.5), 29287689 (0.5:0.75), 26964672 (0.75:1) MAX=75348463

iii


In [10]:
def get_wav(file):
   waveform, sr = torchaudio.load(file); waveform = waveform[0]
   return len(waveform)

df_toy = df[:int(0.25*len(df))]
df['dd'] = df['relative_path'].apply(get_wav)

mid = []
for i in range(len(df)):
  mid.append(df.iloc[i,-1])
print(np.mean(np.array(mid))) 

1937318.8119939747


In [13]:
def get_mrcg_from_file(file):
  
  mid = 1937318 # zero padding so that the length is equal to the length of the longest waveform 
  print("iii")
  waveform, sr = torchaudio.load(file)
  if len(waveform[0]) < mid:
    target = torch.zeros(mid)
    source = waveform[0]
    target[:len(source)] = source
    return mrcg_extract(target)
  else:
    waveform = waveform[0]
    return mrcg_extract(waveform[:mid])

In [15]:
df_toy = df[:2]
df_toy['dd'] = df_toy['relative_path'].apply(get_mrcg_from_file)

df_toy

#df_train['mrcg'] = df_train['relative_path'].apply(get_mrcg_from_file)

iii
iii


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,relative_path,primary_label,dd
1466,/content/tmp/amerob/XC107523.ogg,0,"[[-1.8068877959212895, -0.275449599027982, 0.9..."
1467,/content/tmp/amerob/XC112597.ogg,0,"[[-2.396268977063757, 0.18771316777299862, 0.8..."


In [8]:
df_toy = df[int(0.25*len(df)):int(0.5*len(df))]
df_toy['dd'] = df_toy['relative_path'].apply(get_mrcg_from_file)

mid = []
for i in range(len(df_toy)):
  mid.append(df_toy.iloc[i,-1])
print(np.mean(np.array(mid))) 

df_toy = df[int(0.5*len(df)):int(0.75*len(df))]
df_toy['dd'] = df_toy['relative_path'].apply(get_mrcg_from_file)

mid = []
for i in range(len(df_toy)):
  mid.append(df_toy.iloc[i,-1])
print(np.mean(np.array(mid))) 

df_toy = df[int(0.75*len(df)):]
df_toy['dd'] = df_toy['relative_path'].apply(get_mrcg_from_file)

mid = []
for i in range(len(df_toy)):
  mid.append(df_toy.iloc[i,-1])
print(np.mean(np.array(mid))) 

KeyboardInterrupt: ignored

In [ ]:
print()

In [ ]:
df_toy[''].value_counts()

In [ ]:
df_toy = df[int(0.25*len(df)):int(0.5*len(df))]
df_toy['mrcg'] = df_toy['relative_path'].apply(get_mrcg_from_file)
max = 0
for i in range(len(df_toy)):
  if max < df_toy.iloc[i,-1].shape[0]:
    max = df_toy.iloc[i,-1].shape[0]
print(max) # 73084832 (0:0.25, ), 75348463 (0.25:0.5), 29287689 (0.5:0.75), 26964672 (0.75:1) MAX=75348463


df_toy = df[int(0.5*len(df)):int(0.75*len(df))]
df_toy['mrcg'] = df_toy['relative_path'].apply(get_mrcg_from_file)
max = 0
for i in range(len(df_toy)):
  if max < df_toy.iloc[i,-1].shape[0]:
    max = df_toy.iloc[i,-1].shape[0]
print(max) # 73084832 (0:0.25, )


df_toy = df[int(0.75*len(df)):]
df_toy['mrcg'] = df_toy['relative_path'].apply(get_mrcg_from_file)
max = 0
for i in range(len(df_toy)):
  if max < df_toy.iloc[i,-1].shape[0]:
    max = df_toy.iloc[i,-1].shape[0]
print(max) # 73084832 (0:0.25, )

In [21]:
df_toy.to_csv('mrcgdata.csv')
!cp mrcgdata.csv "drive/My Drive/"

(768, 2328)

In [47]:
class AudioUtil():
  """
  This class will be for various functions that I will use for the audio data
  """
  @staticmethod
  def open(audio_file):
    """
    The load method loads the audio file and produces 2 outputs:
    - waveform (Tensor): A waveform is a graphical representation of a sound wave as it moves through a medium over time
    - sample rate (int): In audio production, a sample rate defines how many times per second a sound is sampled. Technically speaking, it is the frequency of samples used in a digital recording.
    The standard sample rate used for audio CDs is 44.1 kilohertz (44,100 hertz). That means each second of a song on a CD contains 44,100 individual samples.
    """
    waveform, samplerate = torchaudio.load(audio_file) # https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html#loading-audio-data-into-tensor
    return (waveform, samplerate) # waveform is the PCM data.
  
  # staticmethod
  # def rechannel(aud, new_channel):
  #   sig, sr = aud

  #   if (sig.shape[0] == new_channel):
  #     # Nothing to do
  #     return aud

  #   if (new_channel == 1):
  #     # Convert from stereo to mono by selecting only the first channel
  #     resig = sig[:1, :]
  #   else:
  #     # Convert from mono to stereo by duplicating the first channel
  #     resig = torch.cat([sig, sig])

  #   return ((resig, sr))

  # staticmethod
  # def resample(aud, newsr):
  #   """
  #   This method will make sure that my sampling rate is equal across all audio signals.
  #   I don't think I need this method because I think I have the same sampling rate across all audio signals. 
  #   """
  #   waveform, samplerate = aud # 

  #   if (samplerate == newsr):
  #     # Nothing to do
  #     return aud

  #   num_channels = waveform.shape[0]
  #   # Resample first channel
  #   resig = torchaudio.transforms.Resample(samplerate, newsr)(waveform[:1,:])

  #   ### Don't think I need this because I think I have mono channels across all audio signals
  #   # if (num_channels > 1):
  #   #   # Resample the second channel and merge both channels
  #   #   retwo = torchaudio.transforms.Resample(samplerate, newsr)(waveform[1:,:])
  #   #   resig = torch.cat([resig, retwo])

  #   return ((resig, newsr))

    @staticmethod
    def get_mrcg_from_file(file):
      waveform, sr = torchaudio.load(file)
      MAX = 75348463 # zero padding so that the length is equal to the length of the longest waveform 
      waveform, sr = torchaudio.load(file)
      target = torch.zeros(MAX)
      source = waveform[0]
      target[:len(source)] = source
      return target

  # @ staticmethod
  # def pad_trunc(aud, max_ms):
  #   sig, sr = aud
  #   num_rows, sig_len = sig.shape
  #   max_len = sr//1000 * max_ms

  #   if (sig_len > max_len):
  #     # Truncate the signal to the given length
  #     sig = sig[:,:max_len]

  #   elif (sig_len < max_len):
  #     # Length of padding to add at the beginning and end of the signal
  #     pad_begin_len = random.randint(0, max_len - sig_len)
  #     pad_end_len = max_len - sig_len - pad_begin_len

  #     # Pad with 0s
  #     pad_begin = torch.zeros((num_rows, pad_begin_len))
  #     pad_end = torch.zeros((num_rows, pad_end_len))

  #     sig = torch.cat((pad_begin, sig, pad_end), 1)
      
  #   return (sig, sr)



  # @ staticmethod
  # def time_shift(aud, shift_limit):
  #   sig,sr = aud
  #   _, sig_len = sig.shape
  #   shift_amt = int(random.random() * shift_limit * sig_len)
  #   return (sig.roll(shift_amt), sr)
  
  # @ staticmethod
  # def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
  #   sig,sr = aud
  #   top_db = 80

  #   # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
  #   spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

  #   # Convert to decibels
  #   spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
  #   return (spec)

  # @ staticmethod
  # def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
  #   _, n_mels, n_steps = spec.shape
  #   mask_value = spec.mean()
  #   aug_spec = spec

  #   freq_mask_param = max_mask_pct * n_mels
  #   for _ in range(n_freq_masks):
  #     aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

  #   time_mask_param = max_mask_pct * n_steps
  #   for _ in range(n_time_masks):
  #     aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

  #   return aug_spec

# Data Loader

In [ ]:

class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # self.data_path = str(data_path)
    # self.duration = 4000
    # self.sr = 22050
    # self.channel = 1
    # self.shift_pct = 0.4
            
  
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    # audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    
    # # Get the Class ID
    # class_id = self.df.loc[idx, 'primary_label']

    # aud = AudioUtil.open(audio_file)
    # # Some sounds have a higher sample rate, or fewer channels compared to the
    # # majority. So make all sounds have the same number of channels and same 
    # # sample rate. Unless the sample rate is the same, the pad_trunc will still
    # # result in arrays of different lengths, even though the sound duration is
    # # the same.
    
    
    # reaud = AudioUtil.resample(aud, self.sr)
    # rechan = AudioUtil.rechannel(reaud, self.channel)

    # dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    # shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    # sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    # aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    
    return df['mrcg'].iloc[idx].to(self.device)

In [49]:
myds = SoundDS(df_toy, "")

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.05)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

# Model

In [ ]:


# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=397)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x

class AudioClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 100)
        self.hidden_fc = nn.Linear(100, 100)
        self.output_fc = nn.Linear(100, output_dim)

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        # x = [batch size, height * width]

        h_1 = F.relu(self.input_fc(x))

        # h_1 = [batch size, 250]

        h_2 = F.relu(self.hidden_fc(h_1))

        # h_2 = [batch size, 100]

        y_pred = self.output_fc(h_2)

        # y_pred = [batch size, output dim]

        return y_pred

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

# Training

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = torch.tensor(data[0]).to(device), torch.tensor(data[1]).to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        #if i % 10 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
    
    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=2   # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)



# Testing

In [ ]:
# ----------------------------
# Inference
# ----------------------------
def inference (model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:
      # Get the input features and target labels, and put them on the GPU
      inputs, labels = data[0].to(device), data[1].to(device)

      # Normalize the inputs
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s

      # Get predictions
      outputs = model(inputs)

      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
    
  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

KeyboardInterrupt: ignored